<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
@File name: model1.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''
import numpy as np
from pandas import DataFrame
from pandas import read_csv
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import sequence
import matplotlib.pyplot as plt

In [0]:
def generate_model(shape):
  
  model = Sequential()
  
  model.add(Dense(32, input_dim=shape, kernel_initializer='uniform', activation='relu', dropout=0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(256, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  print(model.summary())

  return model

  

In [0]:
#LOAD DATA
print("Loading data...")

#dataframe = read_csv('/root/data/New_Half_uHoo_featureDataset.csv')

#TRAIN, VALIDATION, TEST SETS

#GENERATE AND COMPILE MODEL
model = generate_model(train.shape[-1])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#history = model.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

